In [1]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v2.4.5 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [2]:
df = spark.createDataFrame([
  ("Warsaw", 2016, 100),
  ("Warsaw", 2017, 200),
  ("Boston", 2015, 50),
  ("Boston", 2016, 150),
  ("Toronto", 2017, 50)],["city", "year", "amount"])

In [3]:
df.show()

+-------+----+------+
 city|year|amount|
+-------+----+------+
 Warsaw|2016| 100|
 Warsaw|2017| 200|
 Boston|2015| 50|
 Boston|2016| 150|
Toronto|2017| 50|
+-------+----+------+

In [4]:
from pyspark.sql.functions import *
gbycy = df.groupBy("city", "year").agg(sum("amount"))

In [5]:
gbycy.show()

+-------+----+-----------+
 city|year|sum(amount)|
+-------+----+-----------+
 Boston|2015| 50|
 Warsaw|2017| 200|
 Boston|2016| 150|
 Warsaw|2016| 100|
Toronto|2017| 50|
+-------+----+-----------+

In [6]:
gbc = df.groupBy("city").agg(sum("amount").alias( "amount")).select("city", lit(None), "amount")

In [7]:
gbc.show()

+-------+----+------+
 city|NULL|amount|
+-------+----+------+
Toronto|null| 50|
 Warsaw|null| 300|
 Boston|null| 200|
+-------+----+------+

In [8]:
ugbc = gbycy.union(gbc)

In [9]:
display(ugbc)

city,year,sum(amount)
Boston,2015,50
Warsaw,2017,200
Boston,2016,150
Warsaw,2016,100
Toronto,2017,50
Toronto,null,50
Warsaw,null,300
Boston,null,200


In [10]:
ugbc.show()

+-------+----+-----------+
 city|year|sum(amount)|
+-------+----+-----------+
 Boston|2015| 50|
 Warsaw|2017| 200|
 Boston|2016| 150|
 Warsaw|2016| 100|
Toronto|2017| 50|
Toronto|null| 50|
 Warsaw|null| 300|
 Boston|null| 200|
+-------+----+-----------+

In [11]:
person = spark.createDataFrame([
    (0, "Bill Chambers", 0, [100]),
    (1, "Matei Zaharia", 1, [500, 250, 100]),
    (2, "Michael Armbrust", 1, [250, 100])], ["id", "name", "graduate_program", "spark_status"])

In [12]:
graduateProgram = spark.createDataFrame([
    (0, "Masters", "School of Information", "UC Berkeley"),
    (2, "Masters", "EECS", "UC Berkeley"),
    (1, "Ph.D.", "EECS", "UC Berkeley")], ["id", "degree", "department", "school"])

In [13]:
sparkStatus = spark.createDataFrame([
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor")],["id", "status"])

In [14]:
joinexp = (person.graduate_program == graduateProgram.id)
wrjoin = (person.name == graduateProgram.school)

In [15]:
jtype = "inner"

In [16]:
jpg = person.join(graduateProgram, joinexp, jtype)

In [17]:
jpg.show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [18]:
person.show()

+---+----------------+----------------+---------------+
 id| name|graduate_program| spark_status|
+---+----------------+----------------+---------------+
 0| Bill Chambers| 0| [100]|
 1| Matei Zaharia| 1|[500, 250, 100]|
 2|Michael Armbrust| 1| [250, 100]|
+---+----------------+----------------+---------------+

In [19]:
graduateProgram.show()

+---+-------+--------------------+-----------+
 id| degree| department| school|
+---+-------+--------------------+-----------+
 0|Masters|School of Informa...|UC Berkeley|
 2|Masters| EECS|UC Berkeley|
 1| Ph.D.| EECS|UC Berkeley|
+---+-------+--------------------+-----------+

In [20]:
wrjoin = person.join(graduateProgram, wrjoin, jtype)

In [21]:
wrjoin.show()

+---+----+----------------+------------+---+------+----------+------+
 id|name|graduate_program|spark_status| id|degree|department|school|
+---+----+----------------+------------+---+------+----------+------+
+---+----+----------------+------------+---+------+----------+------+

In [22]:
jytpe = "outer"
ojoin = person.join(graduateProgram, joinexp, jytpe)

In [23]:
ojoin.show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
null| null| null| null| 2|Masters| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [24]:
person.join(graduateProgram, joinexp, "left_outer").show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [25]:
person.join(graduateProgram, joinexp, "right_outer").show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
null| null| null| null| 2|Masters| EECS|UC Berkeley|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [26]:
person.join(graduateProgram, joinexp, "cross").show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [27]:
person.crossJoin(graduateProgram).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 id| name|graduate_program| spark_status| id| degree| department| school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
 0| Bill Chambers| 0| [100]| 0|Masters|School of Informa...|UC Berkeley|
 0| Bill Chambers| 0| [100]| 2|Masters| EECS|UC Berkeley|
 0| Bill Chambers| 0| [100]| 1| Ph.D.| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 0|Masters|School of Informa...|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 2|Masters| EECS|UC Berkeley|
 1| Matei Zaharia| 1|[500, 250, 100]| 1| Ph.D.| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 0|Masters|School of Informa...|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 2|Masters| EECS|UC Berkeley|
 2|Michael Armbrust| 1| [250, 100]| 1| Ph.D.| EECS|UC Berkeley|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+

In [28]:
person.withColumnRenamed("id", "personId")\
.join(sparkStatus, expr("array_contains(spark_status, id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
personId| name|graduate_program| spark_status| id| status|
+--------+----------------+----------------+---------------+---+--------------+
 0| Bill Chambers| 0| [100]|100| Contributor|
 1| Matei Zaharia| 1|[500, 250, 100]|500|Vice President|
 1| Matei Zaharia| 1|[500, 250, 100]|250| PMC Member|
 1| Matei Zaharia| 1|[500, 250, 100]|100| Contributor|
 2|Michael Armbrust| 1| [250, 100]|250| PMC Member|
 2|Michael Armbrust| 1| [250, 100]|100| Contributor|
+--------+----------------+----------------+---------------+---+--------------+

In [29]:
sparkStatus

Out[29]: DataFrame[id: bigint, status: string]

In [30]:
sparkStatus.show()

+---+--------------+
 id| status|
+---+--------------+
500|Vice President|
250| PMC Member|
100| Contributor|
+---+--------------+

In [31]:
sdf = spark.read.text("/FileStore/tables/2010_summary-506d8.csv")

In [32]:
display(sdf)

value
"DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count"
"United States,Romania,1"
"United States,Ireland,264"
"United States,India,69"
"Egypt,United States,24"
"Equatorial Guinea,United States,1"
"United States,Singapore,25"
"United States,Grenada,54"
"Costa Rica,United States,477"
"Senegal,United States,29"


In [33]:
sdf

Out[60]: DataFrame[value: string]

In [34]:
display(sdf)

value
"DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count"
"United States,Romania,1"
"United States,Ireland,264"
"United States,India,69"
"Egypt,United States,24"
"Equatorial Guinea,United States,1"
"United States,Singapore,25"
"United States,Grenada,54"
"Costa Rica,United States,477"
"Senegal,United States,29"


In [35]:
sdf.count()

Out[62]: 256

In [36]:
display(sdf.select(col("value")))

value
"DEST_COUNTRY_NAME,ORIGIN_COUNTRY_NAME,count"
"United States,Romania,1"
"United States,Ireland,264"
"United States,India,69"
"Egypt,United States,24"
"Equatorial Guinea,United States,1"
"United States,Singapore,25"
"United States,Grenada,54"
"Costa Rica,United States,477"
"Senegal,United States,29"


In [37]:
cols = sdf.select(split(sdf.value, ',').alias("combinedCol"))

In [38]:
cols.show(10)

+--------------------+
 combinedCol|
+--------------------+
[DEST_COUNTRY_NAM...|
[United States, R...|
[United States, I...|
[United States, I...|
[Egypt, United St...|
[Equatorial Guine...|
[United States, S...|
[United States, G...|
[Costa Rica, Unit...|
[Senegal, United ...|
+--------------------+
only showing top 10 rows

In [39]:
filtered = cols.filter(cols.combinedCol[0] != "DEST_COUNTRY_NAME")

In [40]:
display(filtered)

combinedCol
"List(United States, Romania, 1)"
"List(United States, Ireland, 264)"
"List(United States, India, 69)"
"List(Egypt, United States, 24)"
"List(Equatorial Guinea, United States, 1)"
"List(United States, Singapore, 25)"
"List(United States, Grenada, 54)"
"List(Costa Rica, United States, 477)"
"List(Senegal, United States, 29)"
"List(United States, Marshall Islands, 44)"


In [41]:
sepCols = filtered.select(filtered.combinedCol[0].alias("Destination"), filtered.combinedCol[1].alias("Origin"), filtered.combinedCol[2].alias("Count"))

In [42]:
display(sepCols)

Destination,Origin,Count
United States,Romania,1
United States,Ireland,264
United States,India,69
Egypt,United States,24
Equatorial Guinea,United States,1
United States,Singapore,25
United States,Grenada,54
Costa Rica,United States,477
Senegal,United States,29
United States,Marshall Islands,44
